In [1]:
!git clone https://github.com/MarkusRosen/keras-efficientnet-regression.git

Cloning into 'keras-efficientnet-regression'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 115 (delta 50), reused 68 (delta 19), pack-reused 0
Receiving objects: 100% (115/115), 3.72 MiB | 32.60 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
cd keras-efficientnet-regression/

/content/keras-efficientnet-regression


In [3]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b0.tar.gz

!tar -xf noisy_student_efficientnet-b0.tar.gz

!python efficientnet_weight_update_util.py --model b0 --notop --ckpt noisy_student_efficientnet-b0/model.ckpt --o efficientnetb0_notop.h5

--2021-12-18 19:12:55--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 142.250.107.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156407520 (149M) [application/octet-stream]
Saving to: ‘noisy_student_efficientnet-b0.tar.gz’

noisy_student_effic 100%[===================>] 149.16M  71.9MB/s    in 2.1s    

2021-12-18 19:12:58 (71.9 MB/s) - ‘noisy_student_efficientnet-b0.tar.gz’ saved [156407520/156407520]

2021-12-18 19:13:05.338388: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
16719872/16705208 [==============================] - 0s 0us/step
check variables match in each bloc

In [4]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 8.0 MB/s 


In [5]:
from tensorflow.keras.losses import MeanAbsoluteError

In [6]:
from tensorflow.keras.losses import MeanSquaredError

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History

import tensorflow_addons as tfa

In [9]:
!pip install -r requirements.txt

     |████████████████████████████████| 11.6 MB 11.4 MB/s 
     |████████████████████████████████| 20.8 MB 1.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 10.5 MB 51.4 MB/s 
     |████████████████████████████████| 3.0 MB 30.7 MB/s 
     |████████████████████████████████| 6.8 MB 31.6 MB/s 
     |████████████████████████████████| 283 kB 54.9 MB/s 
ERROR: Could not find a version that satisfies the requirement tensorboard==2.3.1 (from versions: 1.6.0rc0, 1.6.0, 1.7.0, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.0, 1.12.1, 1.12.2, 1.13.0, 1.13.1, 1.14.0, 1.15.0, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.6.0, 2.7.0)
ERROR: No matching distribution found for tensorboard==2.3.1


In [10]:
cd ..

/content


In [11]:
!unzip /content/drive/MyDrive/CVD19Percentage/Train/Train.zip

Archive:  /content/drive/MyDrive/CVD19Percentage/Train/Train.zip
   creating: Train/
  inflating: Train/Image_0001.png    
  inflating: Train/Image_0004.png    
  inflating: Train/Image_0005.png    
  inflating: Train/Image_0007.png    
  inflating: Train/Image_0011.png    
  inflating: Train/Image_0013.png    
  inflating: Train/Image_0014.png    
  inflating: Train/Image_0016.png    
  inflating: Train/Image_0019.png    
  inflating: Train/Image_0026.png    
  inflating: Train/Image_0034.png    
  inflating: Train/Image_0036.png    
  inflating: Train/Image_0038.png    
  inflating: Train/Image_0040.png    
  inflating: Train/Image_0044.png    
  inflating: Train/Image_0045.png    
  inflating: Train/Image_0050.png    
  inflating: Train/Image_0051.png    
  inflating: Train/Image_0052.png    
  inflating: Train/Image_0053.png    
  inflating: Train/Image_0056.png    
  inflating: Train/Image_0061.png    
  inflating: Train/Image_0063.png    
  inflating: Train/Image_0065.png    
  i

In [12]:
!unzip "/content/drive/MyDrive/CVD19Percentage/Val/Val Blind.zip"

Archive:  /content/drive/MyDrive/CVD19Percentage/Val/Val Blind.zip
   creating: Val/
  inflating: Val/Image_0000.png      
  inflating: Val/Image_0001.png      
  inflating: Val/Image_0002.png      
  inflating: Val/Image_0003.png      
  inflating: Val/Image_0004.png      
  inflating: Val/Image_0005.png      
  inflating: Val/Image_0007.png      
  inflating: Val/Image_0010.png      
  inflating: Val/Image_0011.png      
  inflating: Val/Image_0012.png      
  inflating: Val/Image_0013.png      
  inflating: Val/Image_0014.png      
  inflating: Val/Image_0016.png      
  inflating: Val/Image_0017.png      
  inflating: Val/Image_0018.png      
  inflating: Val/Image_0019.png      
  inflating: Val/Image_0020.png      
  inflating: Val/Image_0021.png      
  inflating: Val/Image_0022.png      
  inflating: Val/Image_0024.png      
  inflating: Val/Image_0026.png      
  inflating: Val/Image_0027.png      
  inflating: Val/Image_0029.png      
  inflating: Val/Image_0030.png      
  i

In [13]:
df = pd.read_csv("/content/Train.csv")

In [14]:
df.head()

,Image_0000.png,0.0,0
0,Image_0001.png,0.0,0
1,Image_0002.png,0.0,0
2,Image_0003.png,0.0,0
3,Image_0004.png,0.0,0
4,Image_0005.png,0.0,0


In [15]:
import os

In [16]:
df_tost = pd.DataFrame(os.listdir("/content/Val"),columns = ['zpid'])

In [17]:

df.columns = ['zpid','label','Subject']

In [18]:
# with open("/content/keras-efficientnet-regression/df.pkl", "rb") as fh:
#   df = pickle.load(fh)
# # df = pd.read_pickle("/content/keras-efficientnet-regression/df.pkl")
# df["image_location"] = (
#     "/content/keras-efficientnet-regression/processed_images/" + df["zpid"] + ".png"
# )  # add the correct path for the image locations.
# if False == True:
#     df = df.iloc[0:1000] 

In [19]:
df["image_location"] = (
    "/content/Train/" + df["zpid"]) 

In [20]:
df_tost["image_location"] = (
    "/content/Val/" + df_tost["zpid"]) 

In [21]:
df.head()

,zpid,label,Subject,image_location
0,Image_0001.png,0.0,0,/content/Train/Image_0001.png
1,Image_0002.png,0.0,0,/content/Train/Image_0002.png
2,Image_0003.png,0.0,0,/content/Train/Image_0003.png
3,Image_0004.png,0.0,0,/content/Train/Image_0004.png
4,Image_0005.png,0.0,0,/content/Train/Image_0005.png


In [22]:
# def run():
#     ...
#     train, val, test = split_data(df)  # split your data
#     mean_baseline = get_mean_baseline(train, val)


In [23]:
def split_data(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Accepts a Pandas DataFrame and splits it into training, testing and validation data. Returns DataFrames.

    Parameters
    ----------
    df : pd.DataFrame
        Your Pandas DataFrame containing all your data.

    Returns
    -------
    Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]
        [description]
    """
    train, val = train_test_split(df, test_size=0.2, random_state=1)  # split the data with a validation size o 20%
    train, test = train_test_split(
        train, test_size=0.125, random_state=1
    )  # split the data with an overall  test size of 10%

    print("shape train: ", train.shape)  # type: ignore
    print("shape val: ", val.shape)  # type: ignore
    print("shape test: ", test.shape)  # type: ignore

    print("Descriptive statistics of train:")
    print(train.describe())  # type: ignore
    return train, val, test  # type: ignore

In [24]:
def get_mean_baseline(train: pd.DataFrame, val: pd.DataFrame) -> float:
    """Calculates the mean MAE and MAPE baselines by taking the mean values of the training data as prediction for the
    validation target feature.

    Parameters
    ----------
    train : pd.DataFrame
        Pandas DataFrame containing your training data.
    val : pd.DataFrame
        Pandas DataFrame containing your validation data.

    Returns
    -------
    float
        MAPE value.
    """
    y_hat = train["label"].mean()
    val["y_hat"] = y_hat
    mae = MeanAbsoluteError()
    mae = mae(val["label"], val["y_hat"]).numpy()  # type: ignore
    mape = MeanAbsolutePercentageError()
    mape = mape(val["label"], val["y_hat"]).numpy()  # type: ignore

    print(mae)
    print("mean baseline MAPE: ", mape)

    return mape

In [25]:
train, val, test = split_data(df)  # split your data
mean_baseline = get_mean_baseline(train, val)

shape train:  (2136, 4)
shape val:  (611, 4)
shape test:  (306, 4)
Descriptive statistics of train:
             label      Subject
count  2136.000000  2136.000000
mean     18.011704    62.073502
std      25.484722    39.473760
min       0.000000     0.000000
25%       1.000000    26.000000
50%       6.000000    63.000000
75%      25.000000    97.000000
max     100.000000   131.000000
19.438369750976562
mean baseline MAPE:  4156547328.0


In [26]:
def create_generators(
    df: pd.DataFrame, train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame,df_tost:pd.DataFrame, visualize_augmentations=False) -> Tuple[Iterator, Iterator, Iterator]:
    """Accepts four Pandas DataFrames: all your data, the training, validation and test DataFrames. Creates and returns
    keras ImageDataGenerators. Within this function you can also visualize the augmentations of the ImageDataGenerators.

    Parameters
    ----------
    df : pd.DataFrame
        Your Pandas DataFrame containing all your data.
    train : pd.DataFrame
        Your Pandas DataFrame containing your training data.
    val : pd.DataFrame
        Your Pandas DataFrame containing your validation data.
    test : pd.DataFrame
        Your Pandas DataFrame containing your testing data.

    Returns
    -------
    Tuple[Iterator, Iterator, Iterator]
        keras ImageDataGenerators used for training, validating and testing of your models.
    """
    train_generator = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=5,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # brightness_range=(0.75, 1),
        # shear_range=0.1,
        # zoom_range=[0.75, 1],
        horizontal_flip=False,
        validation_split=0.2,
    )  # create an ImageDataGenerator with multiple image augmentations
    validation_generator = ImageDataGenerator(
        rescale=1.0 / 255
    )  # except for rescaling, no augmentations are needed for validation and testing generators
    test_generator = ImageDataGenerator(rescale=1.0 / 255)
    real_tost_generator = ImageDataGenerator(rescale=1.0 / 255)
    # visualize image augmentations
    if visualize_augmentations == True:
        visualize_augmentations(train_generator, df)

    train_generator = train_generator.flow_from_dataframe(
        dataframe=train,
        x_col="image_location",  # this is where your image data is stored
        y_col="label",  # this is your target feature
        class_mode="raw",  # use "raw" for regressions
        target_size=(448, 448),
        batch_size=32, # increase or decrease to fit your GPU
    )

    validation_generator = validation_generator.flow_from_dataframe(
        dataframe=val,
        x_col="image_location",
        y_col="label",
        class_mode="raw",
        target_size=(448, 448),
        batch_size=32,
    )
    test_generator = test_generator.flow_from_dataframe(
        dataframe=test,
        x_col="image_location",
        y_col="label",
        class_mode="raw",
        target_size=(448, 448),
        batch_size=32,
    )
    rea_test_generator = real_tost_generator.flow_from_dataframe(
        dataframe=df_tost,
        x_col="image_location",
        y_col=None,
        class_mode=None,
        target_size=(448, 448),
        batch_size=32,
        shuffle=False
    )
    print(rea_test_generator.filenames)
    return train_generator, validation_generator, test_generator, rea_test_generator

In [27]:
def visualize_augmentations(data_generator: ImageDataGenerator, df: pd.DataFrame,visualize_augmentations=False):
    """Visualizes the keras augmentations with matplotlib in 3x3 grid. This function is part of create_generators() and
    can be accessed from there.

    Parameters
    ----------
    data_generator : Iterator
        The keras data generator of your training data.
    df : pd.DataFrame
        The Pandas DataFrame containing your training data.
    """
    # super hacky way of creating a small dataframe with one image
    series = df.iloc[2]

    df_augmentation_visualization = pd.concat([series, series], axis=1).transpose()

    iterator_visualizations = data_generator.flow_from_dataframe(  # type: ignore
        dataframe=df_augmentation_visualization,
        x_col="image_location",
        y_col="label",
        class_mode="raw",
        target_size=(448, 448),  # size of the image
        batch_size=1,  # use only one image for visualization
    )

    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)  # create a 3x3 grid
        batch = next(iterator_visualizations)  # get the next image of the generator (always the same image)
        img = batch[0]  # type: ignore
        img = img[0, :, :, :]  # remove one dimension for plotting without issues
        plt.imshow(img)
    plt.show()
    plt.close()

In [28]:
train_generator, validation_generator, test_generator,real_tost_generator = create_generators(
    df=df, train=train, val=val, test=test,df_tost=df_tost)

Found 2136 validated image filenames.
Found 611 validated image filenames.
Found 306 validated image filenames.
Found 1301 validated image filenames.
['/content/Val/Image_1223.png', '/content/Val/Image_1212.png', '/content/Val/Image_0584.png', '/content/Val/Image_0700.png', '/content/Val/Image_1037.png', '/content/Val/Image_1141.png', '/content/Val/Image_1084.png', '/content/Val/Image_0010.png', '/content/Val/Image_0980.png', '/content/Val/Image_0939.png', '/content/Val/Image_0920.png', '/content/Val/Image_0499.png', '/content/Val/Image_0478.png', '/content/Val/Image_0450.png', '/content/Val/Image_1109.png', '/content/Val/Image_0265.png', '/content/Val/Image_0843.png', '/content/Val/Image_0287.png', '/content/Val/Image_0995.png', '/content/Val/Image_0785.png', '/content/Val/Image_0585.png', '/content/Val/Image_0453.png', '/content/Val/Image_1017.png', '/content/Val/Image_0633.png', '/content/Val/Image_0086.png', '/content/Val/Image_0073.png', '/content/Val/Image_1019.png', '/content/Va

In [29]:


# def run():
#     ...
#     train_generator, validation_generator, test_generator = create_generators(
#         df=df, train=train, val=val, test=test, visualize_augmentations=True
#     )

In [30]:
tfa.optimizers.SGDW

tensorflow_addons.optimizers.weight_decay_optimizers.SGDW

In [31]:
def run_model(
    model_name: str,
    model_function: Model,
    lr: float,
    train_generator: Iterator,
    validation_generator: Iterator,
    test_generator: Iterator,
) -> History:
    """This function runs a keras model with the Ranger optimizer and multiple callbacks. The model is evaluated within
    training through the validation generator and afterwards one final time on the test generator.

    Parameters
    ----------
    model_name : str
        The name of the model as a string.
    model_function : Model
        Keras model function like small_cnn()  or adapt_efficient_net().
    lr : float
        Learning rate.
    train_generator : Iterator
        keras ImageDataGenerators for the training data.
    validation_generator : Iterator
        keras ImageDataGenerators for the validation data.
    test_generator : Iterator
        keras ImageDataGenerators for the test data.

    Returns
    -------
    History
        The history of the keras model as a History object. To access it as a Dict, use history.history. For an example
        see plot_results().
    """

    callbacks = get_callbacks(model_name)
    model = model_function
    model.summary()
    plot_model(model, to_file=model_name + ".jpg", show_shapes=True)
    # radam = tfa.optimizers.SGDW(momentum=0.9,learning_rate=0.0001,weight_decay=0.1)
    radam = tfa.optimizers.RectifiedAdam(learning_rate=lr)
    ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
    optimizer = ranger

    model.compile(
        optimizer=optimizer, loss=MeanAbsoluteError(), metrics=['mae']
    )
    history = model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=callbacks,
        workers=6, # adjust this according to the number of CPU cores of your machine
    )

    model.evaluate(
        test_generator,
        callbacks=callbacks,
    )
    return history  # type: ignore

In [32]:
def get_callbacks(model_name: str) -> List[Union[TensorBoard, EarlyStopping, ModelCheckpoint]]:
    """Accepts the model name as a string and returns multiple callbacks for training the keras model.

    Parameters
    ----------
    model_name : str
        The name of the model as a string.

    Returns
    -------
    List[Union[TensorBoard, EarlyStopping, ModelCheckpoint]]
        A list of multiple keras callbacks.
    """
    logdir = (
        "logs/scalars/" + model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
    )  # create a folder for each model.
    tensorboard_callback = TensorBoard(log_dir=logdir)
    # use tensorboard --logdir logs/scalars in your command line to startup tensorboard with the correct logs

    early_stopping_callback = EarlyStopping(
        monitor="val_mae",
        min_delta=1,  # model should improve by at least 1%
        patience=10,  # amount of epochs  with improvements worse than 1% until the model stops
        verbose=2,
        mode="min",
        restore_best_weights=True,  # restore the best model with the lowest validation error
    )


    model_checkpoint_callback = ModelCheckpoint(
        "./data/models/" + model_name + ".h5",
        monitor="val_mae",
        verbose=0,
        save_best_only=True,  # save the best model
        mode="min",
        save_freq="epoch",  # save every epoch
    )  # saving eff_net takes quite a bit of time
    return [tensorboard_callback, early_stopping_callback, model_checkpoint_callback]


In [36]:


def small_cnn() -> Sequential:
    """A very small custom convolutional neural network with image input dimensions of 224x224x3.

    Returns
    -------
    Sequential
        The keras Sequential model.
    """

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(448, 448, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    # model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(1))

    return model

In [37]:
small_cnn_history = run_model(
        model_name="small_cnn",
        model_function=small_cnn(),
        lr=0.0001,
        train_generator=train_generator,
        validation_generator=validation_generator,
        test_generator=test_generator,
    )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 446, 446, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 223, 223, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 221, 221, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 110, 110, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 flatten (Flatten)           (None, 746496)            0

In [52]:

# def run():
#     ...
#     small_cnn_history = run_model(
#         model_name="small_cnn",
#         model_function=small_cnn(),
#         lr=0.001,
#         train_generator=train_generator,
#         validation_generator=validation_generator,
#         test_generator=test_generator,
#     )

In [33]:
!pip install git+https://github.com/qubvel/classification_models.git


  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-fzxqgnf1
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-fzxqgnf1
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 1.8 MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20045 sha256=42996224c1083d05ba654da322dc41cfa973f9faf0ec0ca8eba5456e32bd5796
  Stored in directory: /tmp/pip-ephem-wheel-cache-rzvf3z2z/wheels/0b/96/56/27b17c903efc647c51e4f364bfc20aa67f8d3dccad63c4fb4e
Successfully built image-classifiers


In [34]:
# for keras
from classification_models.keras import Classifiers

# for tensorflow keras
from classification_models.tfkeras import Classifiers

Classifiers.models_names()

['resnet18',
 'resnet34',
 'resnet50',
 'resnet101',
 'resnet152',
 'seresnet18',
 'seresnet34',
 'seresnet50',
 'seresnet101',
 'seresnet152',
 'seresnext50',
 'seresnext101',
 'senet154',
 'resnet50v2',
 'resnet101v2',
 'resnet152v2',
 'resnext50',
 'resnext101',
 'vgg16',
 'vgg19',
 'densenet121',
 'densenet169',
 'densenet201',
 'inceptionresnetv2',
 'inceptionv3',
 'xception',
 'nasnetlarge',
 'nasnetmobile',
 'mobilenet',
 'mobilenetv2']

In [35]:
ResNeXt50, preprocess_input = Classifiers.get('resnext50')


In [38]:
def adapt_resnext_net() -> Model:
    """This code uses adapts the most up-to-date version of EfficientNet with NoisyStudent weights to a regression
    problem. Most of this code is adapted from the official keras documentation.

    Returns
    -------
    Model
        The keras model.
    """
    inputs = layers.Input(
        shape=(448, 448, 3)
    )  # input shapes of the images should always be 224x224x3 with EfficientNetB0
    # use the downloaded and converted newest EfficientNet wheights
    model = ResNeXt50(include_top = False, input_tensor=inputs, weights='imagenet')
    # model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="/content/keras-efficientnet-regression/efficientnetb0_notop.h5")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1,'relu',name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")

    return model

In [ ]:
def adapt_efficient_net() -> Model:
    """This code uses adapts the most up-to-date version of EfficientNet with NoisyStudent weights to a regression
    problem. Most of this code is adapted from the official keras documentation.

    Returns
    -------
    Model
        The keras model.
    """
    inputs = layers.Input(
        shape=(448, 448, 3)
    )  # input shapes of the images should always be 224x224x3 with EfficientNetB0
    # use the downloaded and converted newest EfficientNet wheights
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="/content/keras-efficientnet-regression/efficientnetb0_notop.h5")
    # Freeze the pretrained weights
    model.trainable = True

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1,'relu',name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")

    return model

In [39]:
RX_net_history = run_model(
        model_name="eff_net",
        model_function=adapt_resnext_net(),
        lr=0.001,
        train_generator=train_generator,
        validation_generator=validation_generator,
        test_generator=test_generator,
    )

94437376/94428600 [==============================] - 1s 0us/step
Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 448, 448, 3)  9           ['input_2[0][0]']                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 454, 454, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)       

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/100
67/67 [==============================] - 71s 683ms/step - loss: 17.9336 - mae: 17.9336 - val_loss: 18.2092 - val_mae: 18.2092
Epoch 2/100
67/67 [==============================] - 40s 580ms/step - loss: 17.7389 - mae: 17.7389 - val_loss: 17.8290 - val_mae: 17.8290
Epoch 3/100
67/67 [==============================] - 39s 569ms/step - loss: 17.2620 - mae: 17.2620 - val_loss: 17.2267 - val_mae: 17.2267
Epoch 4/100
67/67 [==============================] - 40s 580ms/step - loss: 16.7397 - mae: 16.7397 - val_loss: 16.8051 - val_mae: 16.8051
Epoch 5/100
67/67 [==============================] - 39s 568ms/step - loss: 16.4084 - mae: 16.4084 - val_loss: 16.6158 - val_mae: 16.6158
Epoch 6/100
67/67 [==============================] - 39s 569ms/step - loss: 16.2254 - mae: 16.2254 - val_loss: 16.5726 - val_mae: 16.5726
Epoch 7/100
67/67 [==============================] - 39s 567ms/step - loss: 16.1333 - mae: 16.1333 - val_loss: 16.5310 - val_mae: 16.5310
Epoch 8/100
67/67 [===============

In [ ]:
eff_net_history = run_model(
        model_name="eff_net",
        model_function=adapt_efficient_net(),
        lr=0.001,
        train_generator=train_generator,
        validation_generator=validation_generator,
        test_generator=test_generator,
    )

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 448, 448, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 448, 448, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 449, 449, 3)  0           ['normalization[0][0]'

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/100
67/67 [==============================] - 147s 1s/step - loss: 15.2847 - mae: 15.6869 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 2/100
67/67 [==============================] - 70s 1s/step - loss: 7.6775 - mae: 8.0539 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 3/100
67/67 [==============================] - 70s 1s/step - loss: 6.3228 - mae: 6.6992 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 4/100
67/67 [==============================] - 74s 1s/step - loss: 5.3524 - mae: 5.7270 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 5/100
67/67 [==============================] - 73s 1s/step - loss: 5.3954 - mae: 5.7719 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 6/100
67/67 [==============================] - 70s 1s/step - loss: 4.6557 - mae: 5.0273 - val_loss: 17.8903 - val_mae: 18.2750
Epoch 7/100
32/67 [=============>................] - ETA: 32s - loss: 4.8697 - mae: 5.2343

KeyboardInterrupt: ignored

In [ ]:
# def run():
#     ...
#     eff_net_history = run_model(
#         model_name="eff_net",
#         model_function=adapt_efficient_net(),
#         lr=0.5,
#         train_generator=train_generator,
#         validation_generator=validation_generator,
#         test_generator=test_generator,
#     )

In [ ]:
def run():
    ...
    plot_results(small_cnn_history, eff_net_history, mean_baseline)

In [ ]:
def plot_results(model_history_small_cnn: History, model_history_eff_net: History, mean_baseline: float):
    """This function uses seaborn with matplotlib to plot the trainig and validation losses of both input models in an
    sns.relplot(). The mean baseline is plotted as a horizontal red dotted line.

    Parameters
    ----------
    model_history_small_cnn : History
        keras History object of the model.fit() method.
    model_history_eff_net : History
        keras History object of the model.fit() method.
    mean_baseline : float
        Result of the get_mean_baseline() function.
    """

    # create a dictionary for each model history and loss type
    dict1 = {
        "MAPE": model_history_small_cnn.history["mean_absolute_percentage_error"],
        "type": "training",
        "model": "small_cnn",
    }
    dict2 = {
        "MAPE": model_history_small_cnn.history["val_mean_absolute_percentage_error"],
        "type": "validation",
        "model": "small_cnn",
    }
    dict3 = {
        "MAPE": model_history_eff_net.history["mean_absolute_percentage_error"],
        "type": "training",
        "model": "eff_net",
    }
    dict4 = {
        "MAPE": model_history_eff_net.history["val_mean_absolute_percentage_error"],
        "type": "validation",
        "model": "eff_net",
    }

    # convert the dicts to pd.Series and concat them to a pd.DataFrame in the long format
    s1 = pd.DataFrame(dict1)
    s2 = pd.DataFrame(dict2)
    s3 = pd.DataFrame(dict3)
    s4 = pd.DataFrame(dict4)
    df = pd.concat([s1, s2, s3, s4], axis=0).reset_index()
    grid = sns.relplot(data=df, x=df["index"], y="MAPE", hue="model", col="type", kind="line", legend=False)
    grid.set(ylim=(20, 100))  # set the y-axis limit
    for ax in grid.axes.flat:
        ax.axhline(
            y=mean_baseline, color="lightcoral", linestyle="dashed"
        )  # add a mean baseline horizontal bar to each plot
        ax.set(xlabel="Epoch")
    labels = ["small_cnn", "eff_net", "mean_baseline"]  # custom labels for the plot

    plt.legend(labels=labels)
    plt.savefig("training_validation.png")
    plt.show()

In [ ]:
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
import os
from PIL import Image
import numpy as np

# the next 3 lines of code are for my machine and setup due to https://github.com/tensorflow/tensorflow/issues/43174
import tensorflow as tf

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)


def adapt_efficient_net() -> Model:
    """This code uses adapts the most up-to-date version of EfficientNet with NoisyStudent weights to a regression
    problem. Most of this code is adapted from the official keras documentation.

    Returns
    -------
    Model
        The keras model.
    """
    inputs = layers.Input(
        shape=(224, 224, 3)
    )  # input shapes of the images should always be 224x224x3 with EfficientNetB0
    # use the downloaded and converted newest EfficientNet wheights
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="efficientnetb0_notop.h5")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")

    return model


def open_images(inference_folder: str) -> np.ndarray:
    """Loads images from a folder and prepare them for inferencing.

    Parameters
    ----------
    inference_folder : str
        Location of images for inferencing.

    Returns
    -------
    np.ndarray
        List of images as numpy arrays transformed to fit the efficient_net model input specs.
    """
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  # create full path to image

        with Image.open(img_location) as img:  # open image with pillow

            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  # add 0 dimension to fit input shape of efficient_net

        images.append(img)
    images_array = np.vstack(images)  # combine images efficiently to a numpy array
    return images_array


model = adapt_efficient_net()
model.load_weights("./data/models/eff_net.h5")
images = open_images("./inference_samples")

predictions = model.predict(images)

images_names = os.listdir("./inference_samples")
for image_name, prediction in zip(images_names, predictions):
    print(image_name, prediction)

OSError: ignored

In [ ]:
model = small_cnn()
model.load_weights("/content/data/models/small_cnn.h5")
# images = open_images("/content/Val")

predictions = model.predict(real_tost_generator)



In [ ]:
predictions.shape

(1301, 1)

In [ ]:
df_tost

,zpid,image_location,label
0,Image_1223.png,/content/Val/Image_1223.png,0.0
1,Image_1212.png,/content/Val/Image_1212.png,0.0
2,Image_0584.png,/content/Val/Image_0584.png,0.0
3,Image_0700.png,/content/Val/Image_0700.png,0.0
4,Image_1037.png,/content/Val/Image_1037.png,0.0
...,...,...,...
1296,Image_0751.png,/content/Val/Image_0751.png,0.0
1297,Image_0273.png,/content/Val/Image_0273.png,0.0
1298,Image_1161.png,/content/Val/Image_1161.png,0.0
1299,Image_0629.png,/content/Val/Image_0629.png,0.0


In [ ]:
images_names

['Image_1223.png',
 'Image_1212.png',
 'Image_0584.png',
 'Image_0700.png',
 'Image_1037.png',
 'Image_1141.png',
 'Image_1084.png',
 'Image_0010.png',
 'Image_0980.png',
 'Image_0939.png',
 'Image_0920.png',
 'Image_0499.png',
 'Image_0478.png',
 'Image_0450.png',
 'Image_1109.png',
 'Image_0265.png',
 'Image_0843.png',
 'Image_0287.png',
 'Image_0995.png',
 'Image_0785.png',
 'Image_0585.png',
 'Image_0453.png',
 'Image_1017.png',
 'Image_0633.png',
 'Image_0086.png',
 'Image_0073.png',
 'Image_1019.png',
 'Image_0291.png',
 'Image_0525.png',
 'Image_0945.png',
 'Image_0308.png',
 'Image_0824.png',
 'Image_0480.png',
 'Image_0026.png',
 'Image_0672.png',
 'Image_0369.png',
 'Image_0994.png',
 'Image_1259.png',
 'Image_0330.png',
 'Image_0009.png',
 'Image_0846.png',
 'Image_0077.png',
 'Image_0750.png',
 'Image_0432.png',
 'Image_0370.png',
 'Image_0780.png',
 'Image_0104.png',
 'Image_0565.png',
 'Image_0366.png',
 'Image_0747.png',
 'Image_0802.png',
 'Image_1172.png',
 'Image_0886

In [ ]:
ans_dict

{'Image_1223.png': -2.8162067,
 'Image_1212.png': 10.236809,
 'Image_0584.png': 37.566856,
 'Image_0700.png': 13.472925,
 'Image_1037.png': 7.225196,
 'Image_1141.png': 11.137297,
 'Image_1084.png': 2.2347522,
 'Image_0010.png': 0.4477589,
 'Image_0980.png': 7.8047423,
 'Image_0939.png': 4.663079,
 'Image_0920.png': 4.019291,
 'Image_0499.png': 11.327633,
 'Image_0478.png': 5.3832083,
 'Image_0450.png': 15.64106,
 'Image_1109.png': 8.031583,
 'Image_0265.png': 4.008442,
 'Image_0843.png': 27.324486,
 'Image_0287.png': -1.1020896,
 'Image_0995.png': 23.79674,
 'Image_0785.png': 32.678253,
 'Image_0585.png': 42.419285,
 'Image_0453.png': 4.9078617,
 'Image_1017.png': 57.691055,
 'Image_0633.png': 0.35424018,
 'Image_0086.png': 2.2976522,
 'Image_0073.png': 5.216727,
 'Image_1019.png': 41.21726,
 'Image_0291.png': 2.2678478,
 'Image_0525.png': 13.545721,
 'Image_0945.png': 5.562123,
 'Image_0308.png': 1.6459911,
 'Image_0824.png': 8.5436535,
 'Image_0480.png': 7.490985,
 'Image_0026.png':

In [ ]:
df.head(
  
)

,zpid,label,Subject,image_location
0,Image_0001.png,0.0,0,/content/Train/Image_0001.png
1,Image_0002.png,0.0,0,/content/Train/Image_0002.png
2,Image_0003.png,0.0,0,/content/Train/Image_0003.png
3,Image_0004.png,0.0,0,/content/Train/Image_0004.png
4,Image_0005.png,0.0,0,/content/Train/Image_0005.png


In [ ]:
df['label'].value_counts().to_dict()

{0.0: 735,
 1.0: 149,
 2.0: 179,
 3.0: 154,
 4.0: 129,
 5.0: 121,
 6.0: 112,
 7.0: 95,
 8.0: 106,
 9.0: 57,
 10.0: 62,
 11.0: 24,
 12.0: 50,
 13.0: 21,
 14.0: 20,
 15.0: 41,
 16.0: 34,
 17.0: 25,
 18.0: 30,
 19.0: 9,
 20.0: 58,
 21.0: 14,
 22.0: 21,
 23.0: 29,
 24.0: 20,
 25.0: 35,
 26.0: 21,
 27.0: 13,
 28.0: 18,
 29.0: 4,
 30.0: 22,
 31.0: 6,
 32.0: 16,
 33.0: 10,
 34.0: 6,
 35.0: 30,
 36.0: 13,
 37.0: 8,
 38.0: 10,
 39.0: 8,
 40.0: 36,
 41.0: 4,
 42.0: 12,
 43.0: 17,
 44.0: 5,
 45.0: 21,
 46.0: 4,
 47.0: 13,
 48.0: 15,
 49.0: 6,
 50.0: 20,
 51.0: 1,
 52.0: 15,
 53.0: 7,
 54.0: 5,
 55.0: 12,
 56.0: 3,
 57.0: 5,
 58.0: 7,
 60.0: 20,
 61.0: 1,
 62.0: 2,
 63.0: 2,
 64.0: 4,
 65.0: 26,
 66.0: 3,
 67.0: 4,
 68.0: 12,
 69.0: 7,
 70.0: 25,
 71.0: 4,
 72.0: 6,
 73.0: 7,
 75.0: 23,
 76.0: 4,
 77.0: 3,
 78.0: 5,
 79.0: 5,
 80.0: 19,
 81.0: 2,
 82.0: 3,
 83.0: 8,
 84.0: 4,
 85.0: 17,
 86.0: 8,
 87.0: 7,
 88.0: 7,
 89.0: 4,
 90.0: 14,
 91.0: 7,
 92.0: 10,
 93.0: 5,
 94.0: 4,
 95.0: 5,
 96.0: 11,

In [ ]:
round(88.97)

89

In [ ]:
def mapper(i):
  ans = round(i)
  if ans<0:
    ans = 0
  elif ans>100:
    ans = 100
  return float(ans)

In [ ]:
my_dictionary = {k: mapper(v) for k, v in ans_dict.items()}


In [ ]:
my_dictionary

{'Image_1223.png': 0.0,
 'Image_1212.png': 10.0,
 'Image_0584.png': 38.0,
 'Image_0700.png': 13.0,
 'Image_1037.png': 7.0,
 'Image_1141.png': 11.0,
 'Image_1084.png': 2.0,
 'Image_0010.png': 0.0,
 'Image_0980.png': 8.0,
 'Image_0939.png': 5.0,
 'Image_0920.png': 4.0,
 'Image_0499.png': 11.0,
 'Image_0478.png': 5.0,
 'Image_0450.png': 16.0,
 'Image_1109.png': 8.0,
 'Image_0265.png': 4.0,
 'Image_0843.png': 27.0,
 'Image_0287.png': 0.0,
 'Image_0995.png': 24.0,
 'Image_0785.png': 33.0,
 'Image_0585.png': 42.0,
 'Image_0453.png': 5.0,
 'Image_1017.png': 58.0,
 'Image_0633.png': 0.0,
 'Image_0086.png': 2.0,
 'Image_0073.png': 5.0,
 'Image_1019.png': 41.0,
 'Image_0291.png': 2.0,
 'Image_0525.png': 14.0,
 'Image_0945.png': 6.0,
 'Image_0308.png': 2.0,
 'Image_0824.png': 9.0,
 'Image_0480.png': 7.0,
 'Image_0026.png': 1.0,
 'Image_0672.png': 14.0,
 'Image_0369.png': 3.0,
 'Image_0994.png': 11.0,
 'Image_1259.png': 2.0,
 'Image_0330.png': 5.0,
 'Image_0009.png': 0.0,
 'Image_0846.png': 2.0,
 

In [ ]:
sorted(my_dictionary)

In [ ]:
sub = pd.DataFrame(my_dictionary.values(), index=my_dictionary.keys())


In [ ]:
sub

,0
Image_1223.png,0.0
Image_1212.png,10.0
Image_0584.png,38.0
Image_0700.png,13.0
Image_1037.png,7.0
...,...
Image_0751.png,12.0
Image_0273.png,1.0
Image_1161.png,45.0
Image_0629.png,39.0


In [ ]:
sub.sort_index(inplace=True)

In [ ]:
sub

,0
Image_0000.png,5.0
Image_0001.png,2.0
Image_0002.png,2.0
Image_0003.png,13.0
Image_0004.png,27.0
...,...
Image_1296.png,12.0
Image_1297.png,19.0
Image_1298.png,20.0
Image_1299.png,22.0


In [ ]:
sub.to_csv('sumison.csv',header=False)

In [ ]:
ans_dict = dict()
images_names = os.listdir("/content/Val")
for image_name, prediction in zip(images_names, predictions):
  ans_dict[image_name] = prediction[0]
  print(image_name, prediction)

Image_1223.png [-2.8162067]
Image_1212.png [10.236809]
Image_0584.png [37.566856]
Image_0700.png [13.472925]
Image_1037.png [7.225196]
Image_1141.png [11.137297]
Image_1084.png [2.2347522]
Image_0010.png [0.4477589]
Image_0980.png [7.8047423]
Image_0939.png [4.663079]
Image_0920.png [4.019291]
Image_0499.png [11.327633]
Image_0478.png [5.3832083]
Image_0450.png [15.64106]
Image_1109.png [8.031583]
Image_0265.png [4.008442]
Image_0843.png [27.324486]
Image_0287.png [-1.1020896]
Image_0995.png [23.79674]
Image_0785.png [32.678253]
Image_0585.png [42.419285]
Image_0453.png [4.9078617]
Image_1017.png [57.691055]
Image_0633.png [0.35424018]
Image_0086.png [2.2976522]
Image_0073.png [5.216727]
Image_1019.png [41.21726]
Image_0291.png [2.2678478]
Image_0525.png [13.545721]
Image_0945.png [5.562123]
Image_0308.png [1.6459911]
Image_0824.png [8.5436535]
Image_0480.png [7.490985]
Image_0026.png [0.59509075]
Image_0672.png [14.08688]
Image_0369.png [2.8535547]
Image_0994.png [10.701736]
Image_125